In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-04"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10851,2024-05-04,Alemanha Bbl,13:30,Crailsheim Merlins,Bamberg,2.19,1.69,176.5,1.85,1.91,1.5,2.01,1.53,Ik8B4Lbg,0.456621,0.591716,0.540541,0.523560,0.048337,0.2,0.4,NaN,NaN,264.898,79.993726,0.301979,170.138,39.033047,0.229420,192.296,288.700,146.40,205.80,0.0,0.0,0.0,0.0,0.182244,0.022567,0.191758,-1.44,-0.288,-4.131944,0.422972,0.3,-0.122972,-2.90,-0.580,-1.189655,0.392609,0.3,-0.092609
10852,2024-05-04,Austrália Nbl1 Leste,05:00,Norths Bears,Inner West Bulls,2.20,1.62,163.5,1.83,1.87,1.5,2.01,1.47,SdypnWYA,0.454545,0.617284,0.546448,0.534759,0.071829,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,128.64,0.0,0.0,0.0,0.0,0.214724,0.015289,0.219447,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10853,2024-05-04,Austrália Nbl1 Sul,06:30,NW Tasmania,Frankston,1.92,1.80,174.5,1.86,1.84,-3.5,1.98,2.28,tlHTcc83,0.520833,0.555556,0.537634,0.543478,0.076389,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.045620,0.007644,0.099593,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10854,2024-05-04,Austrália Nbl1 Sul,04:30,Nunawading F,Diamond Valley F,2.22,1.61,151.5,1.85,1.85,-1.5,1.95,1.63,lApq1NVb,0.450450,0.621118,0.540541,0.540541,0.071568,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,116.40,0.00,0.0,0.0,0.0,0.0,0.225240,0.000000,0.126410,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10855,2024-05-04,Austrália Nbl1 Sul,05:00,Eltham F,Ringwood F,2.32,1.56,155.5,1.85,1.85,-1.5,1.90,1.80,dzlia10B,0.431034,0.641026,0.540541,0.540541,0.072060,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,134.40,0.0,0.0,0.0,0.0,0.277011,0.000000,0.038222,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10856,2024-05-04,Austrália Nbl1 Oeste,08:00,Warwick Senators,Geraldton Buccaneers,1.95,1.77,190.5,1.84,1.86,1.5,1.83,1.65,nVkwiZA9,0.512821,0.564972,0.543478,0.537634,0.077792,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,146.94,157.04,0.0,0.0,0.0,0.0,0.068430,0.007644,0.073149,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10857,2024-05-04,Austrália Nbl1 Oeste,06:00,East Perth Eagles F,Mandurah Magic F,2.06,1.69,151.5,1.81,1.89,-1.5,1.87,1.79,08S9p0og,0.485437,0.591716,0.552486,0.529101,0.077153,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,278.87,0.00,0.0,0.0,0.0,0.0,0.139536,0.030578,0.030912,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10858,2024-05-04,Espanha Leb Ouro,12:00,Hestia Menorca,Real Bétis,2.01,1.73,155.5,1.85,1.85,1.5,1.87,1.59,pWv1L96k,0.497512,0.578035,0.540541,0.540541,0.075547,0.4,0.8,NaN,NaN,216.810,102.240339,0.471567,143.252,60.883839,0.425012,179.870,258.226,149.40,153.09,0.0,0.0,0.0,0.0,0.105877,0.000000,0.114445,-1.72,-0.344,-2.936047,0.000000,0.0,0.000000,-0.34,-0.068,-10.735294,0.000000,0.0,0.000000
10859,2024-05-04,Filipinas Copa Das Filipinas,04:00,Blackwater Bossing,Phoenix Fuelmasters,1.61,2.25,194.5,1.87,1.93,-4.5,2.05,2.85,80UjlDWd,0.621118,0.444444,0.534759,0.518135,0.065562,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,194.36,153.22,0.0,0.0,0.0,0.0,0.234481,0.022330,0.230892,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10860,2024-05-04,França Lnb,13:30,Nanterre,Dijon,1.77,2.05,161.5,1.81,1.95,-2.5,2.00,2.36,nob6rfnd,0.564972,0.487805,0.552486,0.512821,0.052777,0.4,0.6,NaN,NaN,176.278,28.7

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,06:30,Austrália Nbl1 Sul,NW Tasmania,Frankston,174.5,1.86,1.0000,Over
1,04:00,Filipinas Copa Das Filipinas,Blackwater Bossing,Phoenix Fuelmasters,194.5,1.87,1.0000,Over
2,13:30,França Lnb,Nanterre,Dijon,161.5,1.81,1.0000,Over
3,15:00,França Pro B,Orleans,Boulazac,159.5,1.84,1.0000,Over
4,03:05,Japão B.League,SeaHorses Mikawa,NeoPhoenix,164.5,1.86,1.0000,Over
5,14:00,Letônia Lbl,Rigas Zelli,Ventspils,171.5,1.80,0.6094,Over
6,14:45,Reino Unido Bbl,Leicester Riders,Sheffield Sharks,169.5,1.85,1.0000,Over
7,04:00,Taiwan Wsbl Feminina,Taiyuan F,Cathay Life F,140.5,1.83,1.0000,Over
8,09:30,Turquia Superliga,Manisa,Pinar Karsiyaka,169.5,1.81,1.0000,Over
9,09:30,Turquia Superliga,Tofas,Galatasaray,173.5,1.86,1.0000,Over
